In [136]:
#%pip install pandas pyarrow
import pandas as pd

# Load the Parquet file
articles = pd.read_parquet('articles.parquet', engine='pyarrow')
behaviors = pd.read_parquet('train/behaviors.parquet', engine='pyarrow')
history = pd.read_parquet('train/history.parquet', engine='pyarrow')

In [138]:
articles.head()



,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3037230,Ishockey-spiller: Jeg troede jeg skulle dø,"ISHOCKEY: Ishockey-spilleren Sebastian Harts håber stadig, at karrieren kan fortsætte i USA, eft...",2023-06-29 06:20:57,False,Ambitionerne om at komme til USA og spille ishockey har 21-årige Sebastian Harts stadig.\nMen de...,2003-08-28 08:55:00,None,article_default,https://ekstrabladet.dk/sport/anden_sport/ishockey/article3037230.ece,[],[],"[Kriminalitet, Kendt, Sport, Katastrofe, Mindre ulykke]",142,"[327, 334]",sport,NaN,NaN,NaN,0.9752,Negative
1,3044020,Prins Harry tvunget til dna-test,"Hoffet tvang Prins Harry til at tage dna-test fordi man frygtede, at Dianas elsker, James Hewitt...",2023-06-29 06:21:16,False,Den britiske tabloidavis The Sun fortsætter med at lække historier fra den kommende bog om prins...,2005-06-29 08:47:00,"[3097307, 3097197, 3104927]",article_default,https://ekstrabladet.dk/underholdning/udlandkendte/article3044020.ece,"[Harry, James Hewitt]","[PER, PER]","[Kriminalitet, Kendt, Underholdning, Personfarlig kriminalitet]",414,[432],underholdning,NaN,NaN,NaN,0.7084,Negative
2,3057622,Rådden kørsel på blå plader,Kan ikke straffes: Udenlandske diplomater i Danmark slipper gratis fra spritkørsel og blæser på ...,2023-06-29 06:21:24,False,"Slingrende spritkørsel. Grove overtrædelser af fartbegrænsningerne. Bunker af gamle, ubetalte pa...",2005-10-10 07:20:00,[3047102],article_default,https://ekstrabladet.dk/nyheder/samfund/article3057622.ece,[],[],"[Kriminalitet, Transportmiddel, Bil]",118,[133],nyheder,NaN,NaN,NaN,0.9236,Negative
3,3073151,Mærsk-arvinger i livsfare,FANGET I FLODBØLGEN: Skibsrederens oldebørn måtte have hjælp efter flugt fra strandhotel i Phuket,2023-06-29 06:21:38,False,To oldebørn af skibsreder Mærsk McKinney Møller var i yderste livsfare under juleferien i Phuket...,2005-01-04 06:59:00,"[3067474, 3067478, 3153705]",article_default,https://ekstrabladet.dk/nyheder/samfund/article3073151.ece,[],[],"[Erhverv, Privat virksomhed, Livsstil, Familieliv, Transportmiddel, Kultur, Katastrofe, Mindre u...",118,[133],nyheder,NaN,NaN,NaN,0.9945,Negative
4,3193383,Skød svigersøn gennem babydyne,44-årig kvinde tiltalt for drab på ekssvigersøn sidste år og for at have medvirket til sin mands...,2023-06-29 06:22:57,False,En 44-årig mormor blev i dag fremstillet i et nævningeting i Aalborg tiltalt for drab.\nHun er i...,2003-09-15 15:30:00,None,article_default,https://ekstrabladet.dk/krimi/article3193383.ece,[],[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9966,Negative


In [139]:
behaviors.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,48401,NaN,2023-05-21 21:06:50,21.0,NaN,2,"[9774516, 9771051, 9770028, 9775402, 9774461, 9759544, 9773947, 9142581, 9775331, 9775371, 9759966]",[9759966],22779,False,NaN,NaN,NaN,False,21,16.0,27.0
1,152513,9778745.0,2023-05-24 07:31:26,30.0,100.0,1,"[9778669, 9778736, 9778623, 9089120, 9778661, 9777492, 9778718, 9778657, 9778682, 9482970, 97182...",[9778661],150224,False,NaN,NaN,NaN,False,298,2.0,48.0
2,155390,NaN,2023-05-24 07:30:33,45.0,NaN,1,"[9778369, 9777856, 9778500, 9778021, 9778627, 9778351, 9778155, 9778226, 9777034, 9778628, 9778448]",[9777856],160892,False,NaN,NaN,NaN,False,401,215.0,100.0
3,214679,NaN,2023-05-23 05:25:40,33.0,NaN,2,"[9776715, 9776406, 9776566, 9776071, 9776808, 9776246, 9776497, 9776046, 9776855]",[9776566],1001055,False,NaN,NaN,NaN,False,1357,40.0,47.0
4,214681,NaN,2023-05-23 05:31:54,21.0,NaN,2,"[9775202, 9776855, 9776688, 9771995, 9776583, 9776553, 9695098, 9776071, 9776691, 9776449, 97748...",[9776553],1001055,False,NaN,NaN,NaN,False,1358,5.0,49.0


In [200]:
def process_ids(row):
    clicked_ids = set(row['article_ids_clicked'].split())
    inview_ids = row['article_ids_inview'].split()
    processed_ids = [id + '-1' if id in clicked_ids else id + '-0' for id in inview_ids]
    return ' '.join(processed_ids)

# Convert 'impression_time' to string
behaviors['impression_time'] = behaviors['impression_time'].astype(str)

# Check if 'impression_time' is already in the desired format
if not behaviors['impression_time'].str.contains('/').any():
    # Convert 'impression_time' to datetime
    behaviors['impression_time'] = pd.to_datetime(behaviors['impression_time'])

    # Format 'impression_time' in the desired format
    behaviors['impression_time'] = behaviors['impression_time'].dt.strftime('%m/%d/%Y %I:%M:%S %p')

# Add 'U' before every 'user_id' if it doesn't already start with 'U'
behaviors['user_id'] = behaviors['user_id'].apply(lambda x: 'U' + str(x) if not str(x).startswith('U') else str(x))

# Convert 'article_ids_inview' to string
behaviors['article_ids_inview'] = behaviors['article_ids_inview'].astype(str)


if not behaviors['article_ids_clicked'].str.contains(' N').any():
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].apply(lambda x: 'N' + str(x).split(' ', 1)[0] + ' ' + str(x))
    behaviors['article_ids_clicked'] = behaviors['article_ids_clicked'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

# Check if 'article_ids_inview' is already in the desired format
if not behaviors['article_ids_inview'].str.contains(' N').any():
    # For each line in the 'article_ids_inview' column, add 'N' upfront
    # Add 'N' before each 'article_ids_inview', replace each space with ' N' and remove square brackets
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].apply(lambda x: 'N' + str(x).split(' ', 1)[0] + ' ' + str(x))
    behaviors['article_ids_inview'] = behaviors['article_ids_inview'].str.replace(' ', ' N').str.replace('[', '').str.replace(']', '')

# Select the required columns
train_file = behaviors[['user_id', 'impression_time', 'article_ids_clicked', 'article_ids_inview']]

# Write to a new .tsv file without index
train_file.to_csv('/Users/sebastiaan/Desktop/RecSys-PP-Rec-2/data/Challenge/train.tsv', sep='\t', index=False, header=True)

# Read the file and print the head
df = pd.read_csv('/Users/sebastiaan/Desktop/RecSys-PP-Rec-2/data/Challenge/train.tsv', sep='\t')
df['article_ids_inview'] = df['article_ids_inview'].str.replace('"', '')
df['article_ids_inview'] = df.apply(process_ids, axis=1)

# Write the DataFrame back to the .tsv file
df.to_csv('/Users/sebastiaan/Desktop/RecSys-PP-Rec-2/data/Challenge/train.tsv', sep='\t', index=False, header=False)

df.head()

,user_id,impression_time,article_ids_clicked,article_ids_inview
0,U22779,05/21/2023 09:06:50 PM,N9759966 N9759966,N9774516-0 N9774516-0 N9771051-0 N9770028-0 N9775402-0 N9774461-0 N9759544-0 N9773947-0 N9142581...
1,U150224,05/24/2023 07:31:26 AM,N9778661 N9778661,N9778669-0 N9778669-0 N9778736-0 N9778623-0 N9089120-0 N9778661-1 N9777492-0 N9778718-0 N9778657...
2,U160892,05/24/2023 07:30:33 AM,N9777856 N9777856,N9778369-0 N9778369-0 N9777856-1 N9778500-0 N9778021-0 N9778627-0 N9778351-0 N9778155-0 N9778226...
3,U1001055,05/23/2023 05:25:40 AM,N9776566 N9776566,N9776715-0 N9776715-0 N9776406-0 N9776566-1 N9776071-0 N9776808-0 N9776246-0 N9776497-0 N9776046...
4,U1001055,05/23/2023 05:31:54 AM,N9776553 N9776553,N9775202-0 N9775202-0 N9776855-0 N9776688-0 N9771995-0 N9776583-0 N9776553-1 N9695098-0 N9776071...


In [225]:
import os
import numpy as np
import random
from datetime import datetime

def trans2tsp(date):
    return int(datetime.strptime(date, '%m/%d/%Y %I:%M:%S %p').timestamp())

######################################################################################################################################################

anchor = trans2tsp('10/10/2019 11:59:59 PM')
def parse_time_bucket(date):
    tsp = trans2tsp(date)
    tsp = tsp - anchor
    tsp = tsp//3600
    return tsp

def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

class UserContent():
    def __init__(self,news_index,config,filename,flag):
        self.flag = flag
        self.news_index = news_index
        self.config = config
        self.filename = filename
        self.load_session()
        self.parse_user()

    def load_session(self,):
        config = self.config
        path = config['data_root_path']
        with open(os.path.join(path,self.filename)) as f:
            lines=f.readlines()
     
        session=[]
        for i in range(len(lines)):
            if self.flag == 2:
                _, clicktime, click, imp = lines[i].strip().split('\t')
            elif self.flag == 1:
                _,click, imp = lines[i].strip().split('\t')

            clicks = click.split('#N#')
            click_ids = []
            click_bucket = []
            for i in range(len(clicks)):
                if clicks[i] == '':
                    continue
                    
                # did, clicktime = clicks[i].split('#TAB#')
                # if did == '':
                #     continue
                bucket = parse_time_bucket(clicktime)
                click_ids.append(clicks[i])
                click_bucket.append(bucket)

            imp_list = imp.split(' ')
            pos = []
            neg = []
            for imp_id in imp_list:
                if imp_id[-1] == '1':
                    pos.append(imp_id)
                if imp_id[-1] == '0':
                    neg.append(imp_id)

            # pos, neg, tm = imp.split('#TAB#')
            # bucket = parse_time_bucket(tm)
            # pos = pos.split()
            # neg = neg.split()
            
            session.append([click_ids,click_bucket,bucket,pos,neg])

        print(session[:10])

        self.session = session

    def parse_user(self,):
        session = self.session
        config = self.config

        MAX_ALL = config['max_clicked_news']
        news_index = self.news_index

        user_num = len(session)
        self.click = np.zeros((user_num,MAX_ALL),dtype='int32')
        self.click_bucket = np.zeros((user_num,MAX_ALL),dtype='int32')

        for user_id in range(len(session)):
            click_ids,click_bucket,bucket,pos,neg =session[user_id]
            clicks = []
            for i in range(len(click_ids)):
                clicks.append(news_index[click_ids[i]])

            if len(clicks) >MAX_ALL:
                clicks = clicks[-MAX_ALL:]
                click_bucket = click_bucket[-MAX_ALL:]
            else:
                clicks = [0]*(MAX_ALL-len(click_ids))+clicks
                click_bucket = [1]*(MAX_ALL-len(click_bucket))+click_bucket

                
            self.click[user_id] = np.array(clicks)
            self.click_bucket[user_id] = np.array(click_bucket)

######################################################################################################################################################

# Variables for testing the UserContent.py:
news_index = {"news1": 1, "news2": 2, "news3": 3}
config = {'data_root_path': '/Users/sebastiaan/Desktop/RecSys-PP-Rec-2/data/Challenge','max_clicked_news': 10}
filename = 'train.tsv'
flag = 2  # or 2, depending on your data
user_content = UserContent(news_index, config, filename, flag)

AttributeError: 'list' object has no attribute 'head'

In [10]:
history.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,58608,"[2023-04-27T18:48:09.000000, 2023-04-27T18:48:...","[37.0, 61.0, 100.0, 100.0, 55.0, 100.0, 100.0,...","[9739362, 9739179, 9738567, 9739344, 9739202, ...","[2.0, 24.0, 72.0, 65.0, 11.0, 4.0, 101.0, 0.0,..."
2,95507,"[2023-04-27T15:20:28.000000, 2023-04-27T15:20:...","[60.0, 100.0, 100.0, 21.0, 29.0, 67.0, 49.0, 5...","[9739035, 9738646, 9634967, 9738902, 9735495, ...","[18.0, 29.0, 51.0, 12.0, 10.0, 10.0, 13.0, 24...."
3,106588,"[2023-04-27T08:29:09.000000, 2023-04-27T08:29:...","[24.0, 57.0, 100.0, nan, nan, 100.0, 100.0, 73...","[9738292, 9738216, 9737266, 9737556, 9737657, ...","[9.0, 15.0, 42.0, 9.0, 3.0, 58.0, 26.0, 214.0,..."
4,617963,"[2023-04-27T14:42:25.000000, 2023-04-27T14:43:...","[100.0, 100.0, nan, 46.0, 23.0, 19.0, 61.0, 70...","[9739035, 9739088, 9738902, 9738968, 9738760, ...","[45.0, 29.0, 116.0, 26.0, 34.0, 42.0, 58.0, 59..."


## Create mergeimps.tsv (popularity)
The "mergeimps.tsv'' file needs to store the #click and #impression of each news with time information for building popularity signals. Specifically, we first need to divide the time into several buckets in units of one hour. Thus, since the MIND dataset contains user data in 6 weeks, the time periods can be divided into 2476 buckets. Then, we count the #impressions and #clicks of a news in each bucket and store the results in each line of ``mergeimps.tsv'', where the format is :

[News_ID] TAB [Bucket_ID] TAB #Cliks TAB #Impressions.
 
In MIND, since we did not obtain the click information for test data, which can substantially affect the estimation of popularity signals. Thus, I advise that you can use the validation dataset of MIND for evaluation.

In [ ]:
...

## Create V21UrlDocs22_title_json.tsv ()
For implementing our method based on MIND, the "V21UrlDocs22_title_json.tsv'' needs to store all news in the training/val/test set. Each line stores the information of a single news article, where format is based on 

[News_ID] TAB [Category] TAB [Subcategory] TAB [Title] TAB [Url] TAB [Abstract] TAB [Body].
 
You can modify my codes to avoid the load of some unnecessary news data (e.g., Url, Abstract, and Body)

In [ ]:
...